In [2]:
import mediapipe as mp
import cv2
import tensorflow as tf

import numpy as np
import pathlib
import os

objc[87362]: Class CaptureDelegate is implemented in both /Users/dev/Sign-Language-Translator/env/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x108ed0860) and /Users/dev/Sign-Language-Translator/env/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x17cd2a480). One of the two will be used. Which one is undefined.
objc[87362]: Class CVWindow is implemented in both /Users/dev/Sign-Language-Translator/env/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x107c00a68) and /Users/dev/Sign-Language-Translator/env/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x17cd2a4d0). One of the two will be used. Which one is undefined.
objc[87362]: Class CVView is implemented in both /Users/dev/Sign-Language-Translator/env/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x107c00a90) and /Users/dev/Sign-Language-Translator/env/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x17cd2a4f8). One of the two will be used. Which

In [3]:
mp_holistic = mp.solutions.holistic # get holistic model
mp_drawing = mp.solutions.drawing_utils # get drawing tools

In [4]:
def mp_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # convert image color to RGB
    image.flags.writeable = False                  # don't show this image in feed
    results = model.process(image)                 # make holistics prediction
    image.flags.writeable = True                   # make image writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # convert back to BGR
    return image, results

def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(63)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(63)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
    return np.concatenate([pose, face, lh, rh])

In [5]:
# model.load_weights('models/devins_movinet_v1.h5')

In [6]:
model = tf.saved_model.load('models/')

In [7]:
print(list(model.signatures.keys()))

['serving_default']


In [8]:
infer = model.signatures["serving_default"]
print(infer.structured_outputs)

{'dense_5': TensorSpec(shape=(None, 3), dtype=tf.float32, name='dense_5')}


In [9]:
DATA_PATH = 'experiment_subset'
class_names = sorted(set(p.name for p in pathlib.Path(os.path.join(DATA_PATH, 'train')).iterdir() if p.is_dir()))
label_map = dict((name, idx) for idx, name in enumerate(class_names))
actions = list(label_map.keys())

In [10]:
actions

['hello', 'love', 'thank you']

In [ ]:
sequence = []
sentence = []
out = np.array([0])
threshold = 0.9
num_frames = 12

cap = cv2.VideoCapture(0)
# access mp model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # read feed
        ret, frame = cap.read()
        frame = cv2.resize(frame, (256, 256))

        # get holistic detections
        image, results = mp_detection(frame, holistic)
        
        # draw landmarks
        draw_landmarks(image, results)

        #keypoints = extract_keypoints(results)
        sequence.append(frame)
        #sequence = tf.concat([sequence, frame], 0)
        sequence = sequence[-num_frames:]
        #print(tf.convert_to_tensor(sequence).shape)
        

        if len(sequence) == num_frames:
            #out = model.predict(np.expand_dims(sequence, axis=0))[0]
            #print(actions[np.argmax(out)])
            batch = tf.convert_to_tensor(sequence, dtype = 'float32')
            out = infer(batch[tf.newaxis])['dense_5'].numpy()[0]

        print(out)
        if out[np.argmax(out)] > threshold:
            if len(sentence) > 0:
                if actions[np.argmax(out)] != sentence[-1]:
                    sentence.append(actions[np.argmax(out)])
            else:
                sentence.append(actions[np.argmax(out)])

        if len(sentence) > 5:
            sentence = sentence[-5:]

        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        

        # show to screen
        #cv2.imshow('OpenCV Feed', image)

        # press q to break
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]


2022-10-14 17:58:47.622996: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-10-14 17:58:48.412244: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x120e20550 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-10-14 17:58:48.412259: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Host, Default Version
2022-10-14 17:58:48.418848: I tensorflow/compiler/jit/xla_compilation_cache.cc:476] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
